## Importing libraries

In [7]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## Summarizer

In [13]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


# Setting system prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# Setting user prompt
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

# Putting user and system prompt togehter.
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# This function will give summarization in markdown format.
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

# This function does the job of summmarize function, but it will show the summariztion nicely not in markdown.
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

## An example of using website summarizer

In [17]:
display_summary("https://news.google.com")

**Google News Summary**
===============

*   **General Information**
    *   Google News is the official website of Google's content aggregation service.
*   **Search and Features**
    *   Advanced search functionality provides users with more control over their searches.
    *   Help center guides users on how to use the service effectively.
*   **News Topics**
    *   Current news articles are showcased, including various sections like U.S., World, Business, Technology, Entertainment, Sports, Science, and Health.
    *   A "Trending" section highlights popular stories and topics.

**Recent News Articles**
=====================

### Americas
-   Trump Flees Washington Controversies for Golf-Happy Trip to Scotland (CNN)
-   Security Tightens as Donald Trump Plays Golf at Turnberry
-   Taiwan Opposition Set to Defeat Recall Bid in Blow to Lai Agenda 

### Asia
-   Cambodians flee border with Thailand as clashes continue for third day
-   Thai-Cambodian fighting extends into third day, Malaysia urges ceasefire
-   Cambodia reports first deaths in cross-border conflict with Thailand

### Russia and Global Politics
-   Gabbard and White House 'lying' about intel on Russian interference in 2016, ex-CIA official says